In this notebook we will try to explore the `PySINDy` package architecture, in order to easily implement our bayesian version of this algorithm with a sparsity inducing prior.

We will implement a "custom" `optimizer` module, that will implement a Maximum A Posteriori (_MAP_) algorithm over the distribution derived from data and a (possibly sparsity-inducing) prior distribution.
The main difference is that this object will retain information over the whole probability distribution for the coefficients $\boldsymbol{\xi}$ and not just the best estimates, so that we can evaluate uncertainties over such parameters.

The goal of this algorithm is to compute

$$
P(\boldsymbol{\xi}|\boldsymbol{\dot{u}},\boldsymbol{\Theta}) = P(\boldsymbol{\dot{u}}|\boldsymbol{\xi},\boldsymbol{\Theta})P(\boldsymbol{\xi})
$$

Under the assumption that the likelihood of observing $\boldsymbol{\dot{u}}$ given a certain coefficients vector $\boldsymbol{\xi}$ is a gaussian with mean given by the linear relation:

$$
P(\boldsymbol{\dot{u}}|\boldsymbol{\xi},\boldsymbol{\Theta}) \sim \mathcal{N}(\boldsymbol{\Theta}^T\boldsymbol{\xi},\sigma^2)
$$


And $P(\boldsymbol{\xi})$ will be a sparsity inducing prior, so that the original goal of finding the smallest amount of explanatory terms possible is somewhat obtained.

#### The `BaseOptimizer` class

This is the wrapper class for each optimizer algorithm that the package provides; we will build a optimizer module as a subclass of this wrapper. <a href=https://pysindy.readthedocs.io/en/latest/_modules/pysindy/optimizers/base.html#BaseOptimizer>Source code</a> is available on the documentation.

#### Bayesian Regression implementation

The class will evaluate the best coefficients by performing a Gradient Descent on the posterior distribution, obtained by Bayes theorem with the previously presented likelihood and a prior of choice:

$$
\boldsymbol{\xi}_{best} = \argmin_{\boldsymbol{\xi}}  \left [ - P(\boldsymbol{\xi}|\boldsymbol{\boldsymbol{\dot{u}}},\boldsymbol{\Theta}) \right ] = \argmin_{\boldsymbol{\xi}} \left[ N \log{(\sigma\sqrt{2\pi})}  + \frac{\sum_i^N (\boldsymbol{\dot{u}} - \boldsymbol{\Theta}^T\boldsymbol{\xi} )^2}{2\sigma^2} - \log{P(\boldsymbol{\xi})} \right ]
$$

The initial guess for the coefficients will be the result of a OLS algorithm (already provided by the `BaseOptimizer` class)


In [3]:
from pysindy.optimizers import BaseOptimizer
import numpy as np



class BayesianRegression (BaseOptimizer):
    """
    Bayesian Regression Optimizer.
    
    Evaluates the probability distribution over the coefficients w
    by assuming a data likelihood for y of a gaussian centered at X @ w,
    with a sparsity inducing prior.

    Parameters
    ----------
    fit_intercept : boolean, optional (default False)
        Whether to calculate the intercept for this model. If set to false, no
        intercept will be used in calculations.

    prior : string or custom function (default ???)
        Prior distribution for the coefficients. Defaults to prior inducing???
    
    return_uncertainty : boolean, optional (default False)
        ???? COSA CALCOLIAMO ???

    normalize_columns : boolean, optional (default False)
        Normalize the columns of x (the SINDy library terms) before regression
        by dividing by the L2-norm. Note that the 'normalize' option in sklearn
        is deprecated in sklearn versions >= 1.0 and will be removed.

    copy_X : boolean, optional (default True)
        If True, X will be copied; else, it may be overwritten.
    
    lr : float, optional (default 0.1)
        Learning rate for the gradient descent. 

    tol : float, optional (default 1e-5)
        Tolerance used for determining convergence of the optimization
        algorithm.

    max_iter : int, optional (default 100)
        Maximum iterations of the optimization algorithm.
    
    verbose : boolean, optional (default False)
        enables verbose
    
    sigma : float, optional (default 1) ----> IDEA: DEFAULT TO RESIDUAL SUM OF OLS?
        initial guess for the standard eviation of the gaussian distribution for the target value.

    Attributes
    ----------

    coef_ : array, shape (n_features,) or (n_targets,n_features)
        Coefficients vector.
    
    std_ : ???
    """

    def __init__(
        self, 
        max_iter=20, 
        normalize_columns=False, 
        fit_intercept=False, 
        initial_guess=None, 
        copy_X=True,
        prior='spikeslab',
        lr=0.1,
        tol=1e-5,
        max_iter=100,
        verbose=False
        ):

        # super() calls a temporary version of the parent class
        # this way we pass the init parameters to the class itself via inheritance
        # without having to rewrite everything
        super().__init__(max_iter, normalize_columns, fit_intercept, initial_guess, copy_X)

        self.prior=prior
        self.lr=lr
        self.tol=tol
        if self.max_iter <= 0:
            raise ValueError("Max iteration must be > 0")
        self.verbose=verbose

    def log_posterior(self,coefs,sigma,X,y):
        """
        Logarithm of the posterior distribution.
        """
        n_samples,n_features = X.shape


        a = -n_samples*np.log(sigma*np.sqrt(2*np.pi))
        b =  -np.sum((y - X @ coefs)**2)/(2*sigma**2)
        
        return a + b + np.log(self.prior(coefs))

    
